# Posterior Sampling (PS)

---



今回はPsoterior Sampling(PS)について解説していきます．



以下のコードの例は２つのバンディットのアームがあり，最後にPosterior Samplingのアルゴリズムを使って，２つのアームの平均と分散を予測スロちうアルゴリズムんです．

下のアルゴリズムでは，各ラウンドごとにアームをそれぞれ引き，報酬が高く得られた方を引くということをやっています．

アルゴリズムは以下の流れになります．

1. 各ラウンドでそれぞれアームを引く

2. 報酬が高い方のアームを選び，そのアームの平均と分散を更新していく．



---

In [9]:
import numpy as np


#バンディットクラスの定義
class Bandit:
    def __init__(self,alpha,beta):
        self.alpha = alpha
        self.beta = beta

    def pull(self):
        return np.random.beta(self.alpha,self.beta)
    


#環境の定義
class BayesianBandit:
    def __init__(self):
        self.bandits = [Bandit(2,4),Bandit(5,1)]
        self.samples = [0,0]
        self.mean_estimates = [0,0]
        self.var_estimates = [0,0]


    def choose_bandit(self):
        posterior_samples = [bandit.pull() for bandit in self.bandits]
        bandit_idx = np.argmax(posterior_samples)
        self.samples[bandit_idx] += 1

        #update mean and variance estimates
        reward = self.bandits[bandit_idx].pull()
        prev_mean = self.mean_estimates[bandit_idx]
        prev_var = self.var_estimates[bandit_idx]

        new_mean = (prev_mean * self.samples[bandit_idx] + reward) / (self.samples[bandit_idx] + 1)
        new_var = ((prev_var + prev_mean ** 2) * self.samples[bandit_idx] + reward ** 2) / (self.samples[bandit_idx] + 1) - new_mean ** 2

        self.mean_estimates[bandit_idx] = new_mean
        self.var_estimates[bandit_idx] = new_var

        return bandit_idx
    

bandit = BayesianBandit()

for i in range(100):
    bandit_idx = bandit.choose_bandit()
    
print("Bandit 1 mean estimate: ", bandit.mean_estimates[0])
print("Bandit 1 variance estimate: ", bandit.var_estimates[0])
print("Bandit 2 mean estimate: ", bandit.mean_estimates[1])
print("Bandit 2 variance estimate: ", bandit.var_estimates[1])

print("Bandit 1 samples: ", bandit.samples[0])
print("Bandit 2 samples: ", bandit.samples[1])

Bandit 1 mean estimate:  0.2685841342664545
Bandit 1 variance estimate:  0.039850659403966995
Bandit 2 mean estimate:  0.8433077775656225
Bandit 2 variance estimate:  0.022575731257657772
Bandit 1 samples:  2
Bandit 2 samples:  98


In [12]:
def mean_var(alpha: float, beta: float) :
    mean = alpha / (alpha + beta)
    var = alpha * beta / ((alpha + beta) ** 2 * (alpha + beta + 1))
    return mean, var

print(f'バンディット１の平均と分散: {mean_var(2, 4)}')
print(f'バンディット2の平均と分散: {mean_var(5, 1)}')
print(f'バンディット1の母平均と標本平均の差は{abs(mean_var(2, 4)[0] - 0.26)}')
print(f'バンディット2の母平均と標本平均の差は{abs(mean_var(5, 1)[0] - 0.84)}')
print(f'バンディット１の母分散と標本分散の差は{abs(mean_var(2, 4)[1] - 0.03)}')
print(f'バンディット2の母分散と標本分散の差は{abs(mean_var(5, 1)[1] - 0.02)}')

バンディット１の平均と分散: (0.3333333333333333, 0.031746031746031744)
バンディット2の平均と分散: (0.8333333333333334, 0.01984126984126984)
バンディット1の母平均と標本平均の差は0.0733333333333333
バンディット2の母平均と標本平均の差は0.006666666666666599
バンディット１の母分散と標本分散の差は0.0017460317460317454
バンディット2の母分散と標本分散の差は0.00015873015873016025
